In [1]:
from pickle import load
from numpy import array
from numpy import argmax 
from keras.preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras.models import load_model 
from nltk.translate.bleu_score import corpus_bleu 
from time import time

2023-06-17 19:53:14.308497: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-17 19:53:14.464908: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-17 19:53:14.464924: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-17 19:53:14.965616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-06-1

In [2]:
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

In [3]:
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer 

In [4]:
def max_length(lines):
    return max(len(line.split()) for line in lines)

In [5]:
lines = ['The National Basketball Association (NBA) is a professional basketball league in North America composed of 30 teams', 'It is one of the major professional sports leagues in the United States and Canada and is considered the premier professional basketball league in the world.']
print(lines[0].split())
print(max_length(lines))

['The', 'National', 'Basketball', 'Association', '(NBA)', 'is', 'a', 'professional', 'basketball', 'league', 'in', 'North', 'America', 'composed', 'of', '30', 'teams']
26


In [6]:
def encode_sequences(tokenizer, length, lines):
    X = tokenizer.texts_to_sequences(lines)
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

In [7]:
def word_from_idx(idx, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == idx:
            return word
    return None

In [8]:
dataset = load_clean_sentences('../Data_Cleaning/english-german-both.pkl')
train = load_clean_sentences('../Data_Cleaning/english-german-train.pkl')
test = load_clean_sentences('../Data_Cleaning/english-german-test.pkl')

In [9]:
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print(eng_vocab_size, eng_length)
#
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print(ger_vocab_size, ger_length)

2166 5
3539 8


In [10]:
trainX = encode_sequences(ger_tokenizer, ger_length, train[:,1])
testX  = encode_sequences(ger_tokenizer, ger_length, test[:,1])

In [11]:
print(f'ger_length = {ger_length}')
print(trainX.shape)
print(testX.shape)

ger_length = 8
(9000, 8)
(1000, 8)


In [12]:
source = trainX[0]
print(source)
source = source.reshape((1, source.shape[0]))
print(source)

[ 2 63 26  0  0  0  0  0]
[[ 2 63 26  0  0  0  0  0]]


In [13]:
def predict_sequence(mode, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    indicies = [argmax(vector) for vector in prediction]
    target = list()
    for idx in indicies:
        word = word_from_idx(idx, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    time0 = time()
    for i, source in enumerate(sources):
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predict=[%s]' % (raw_src, raw_target, translation))
        actual.append([raw_target.split()]) 
        predicted.append(translation.split())
        if i == 1000:
            break
             
    print(f'Predict time = {time()-time0}')
    print('BLEU-1 %f' % corpus_bleu(actual, predicted, weights=(1.0, 0.0, 0.0, 0.0)))
    print('BLEU-2 %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0.0, 0.0)))    
    print('BLEU-3 %f' % corpus_bleu(actual, predicted, weights=(0.33, 0.33, 0.33, 0.0))) 
    print('BLEU-4 %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))  
    print(f'BLEU time = {time()-time0}')
    
    
model = load_model('model_1.h5')
evaluate_model(model, eng_tokenizer, trainX, train)    

2023-06-17 19:53:16.175636: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-06-17 19:53:16.175658: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-06-17 19:53:16.175673: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (joseph-VirtualBox): /proc/driver/nvidia/version does not exist
2023-06-17 19:53:16.175863: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
src=[tom will eine], target=[tom wants one], predict=[tom wants one]
src=[nimm meine], target=[take mine], predict=[take mine]
src=[sie wollen mich], target=[they want me], predict=[they want me]
src=[jetzt habe ich es begriffen], target=[i get it now], predict=[i do it now]
src=[fahr sicher], target=[drive safely], predict=[drive safely]
src=[wo ist er], target=[where is it]

In [14]:
evaluate_model(model, eng_tokenizer, testX, test)

src=[was fur eine niete], target=[what a loser], predict=[what a nerve]
src=[horen sie auf zu lacheln], target=[stop smiling], predict=[stop smiling]
src=[die vogel legen eier], target=[birds lay eggs], predict=[the cup]
src=[tom war wutend], target=[tom was angry], predict=[tom was angry]
src=[sie sind wach], target=[theyre awake], predict=[theyre awake]
src=[er schoss auf mich], target=[he shot at me], predict=[he has me]
src=[tom wird laufen], target=[tom will walk], predict=[tom will sing]
src=[ich habe zu tun], target=[im busy], predict=[im not busy]
src=[tom liegt im sterben], target=[tom is dying], predict=[toms dying]
src=[ich habe ferien], target=[im on holiday], predict=[i have up]
Predict time = 40.52018761634827
BLEU-1 0.544372
BLEU-2 0.410727
BLEU-3 0.283821
BLEU-4 0.120561
BLEU time = 40.6274197101593


### The BLEU score usage

reference = list of sentences, where a sentence is a list of tokens <br>
cadidate  = list of sentences

In [15]:
from nltk.translate.bleu_score import sentence_bleu
reference = [['this', 'is', 'a', 'test'], ['this', 'is' 'test']]
candidate = ['this', 'is', 'a', 'test']
score = sentence_bleu(reference, candidate)
print(score)

1.0


reference = list of documents, where a document is a list of sentences <br>
cadidate = list of sentences

In [16]:
# two references for one document
from nltk.translate.bleu_score import corpus_bleu
references = [[['this', 'is', 'a', 'test'], ['this', 'is', 'a', 'test']]]
candidates = [['this', 'is', 'a', 'test']]
score = corpus_bleu(references, candidates)
print(score)

1.0


In [17]:
actual = [["i didnt walk home".split()]]
predicted = ["i didnt walk home".split()]
print(actual)
print(predicted)
print('BLEU-1 %f' % corpus_bleu(actual, predicted, weights=(1.0, 0.0, 0.0, 0.0)))
print('BLEU-2 %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))    
print('BLEU-3 %f' % corpus_bleu(actual, predicted, weights=(0.33, 0.33, 0.33, 0))) 
print('BLEU-4 %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))  

[[['i', 'didnt', 'walk', 'home']]]
[['i', 'didnt', 'walk', 'home']]
BLEU-1 1.000000
BLEU-2 1.000000
BLEU-3 1.000000
BLEU-4 1.000000
